In [61]:
# author: Yifan Gu
# referenced material: https://github.com/maziarraissi/PINNs/tree/master/main
# referenced material: https://github.com/nanditadoloi/PINN

In [62]:
import os
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib import rc
rc('animation', html='jshtml')
import torch
print(torch.cuda.is_available())
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import random
import matplotlib.animation as animation

True


In [63]:
# boundary conditions
trivial = False
constant = False
sinusoidal = True

In [64]:
def nth_derivative(f, wrt, n):
# adapted from the code of Zhuo Shen
# calculate nth derivative of f against wrt
  for i in range(n):
    grads = torch.autograd.grad(f, wrt, create_graph=True, allow_unused=True)[0]
    f = grads.sum()
    if grads is None:
      print('bad grad')
      return torch.tensor(0.)
  return grads

In [65]:
# parameters
rho = 0.5
dpdx = -2.0
g = 0.0
mu = 0.6
nu = mu/rho
h = 1.0
T = 1.0 # max solution time, can change
N = 200
M = 1000
# IC/BC
Vt0 = 0 # time zero; change as needed
def Vlb(t): # lower boundary
  if trivial == True:
    return 0.0
  if constant == True:
    return 0.5
  if sinusoidal == True:
    return np.sin(10*t)
def Vub(t): # upper boundary
  if trivial == True:
    return 0.0
  if constant == True:
    return 1.0
  if sinusoidal == True:
    return np.cos(10*t)
u_t_samples = []
u_y_samples = []
u_samples = []
# sample N points at time zero
for i in range(N):
  u_t_samples.append(0.0)
  u_y_samples.append(random.uniform(0.000000001,h))
  u_samples.append(Vt0)
# sample N points on the lower boundary
for i in range(N):
  t = random.uniform(0.0,T)
  u_t_samples.append(t)
  u_y_samples.append(0.0)
  u_samples.append(Vlb(t))
# sample N points on the upper boundary
for i in range(N):
  t = random.uniform(0.0,T)
  u_t_samples.append(t)
  u_y_samples.append(h)
  u_samples.append(Vub(t))
u_t_samples = np.array(u_t_samples).T
u_y_samples = np.array(u_y_samples).T
u_samples = np.array(u_samples).T
# sample M points on f(t,y)
f_t_samples = []
f_y_samples = []
f_samples = []
for i in range(M):
  f_t_samples.append(np.random.uniform(0.0, T))
  f_y_samples.append(np.random.uniform(0.0, h))
  f_samples.append(0.0)
f_t_samples = np.array(f_t_samples).T
f_y_samples = np.array(f_y_samples).T
f_samples = np.array(f_samples).T
# pack arrays into tensors
u_t_tensor = torch.tensor(u_t_samples).float()
u_t_tensor.requires_grad = True
u_y_tensor = torch.tensor(u_y_samples).float()
u_y_tensor.requires_grad = True
u_tensor = torch.tensor(u_samples).float()
u_tensor.requires_grad = True
f_t_tensor = torch.tensor(f_t_samples).float()
f_t_tensor.requires_grad = True
f_y_tensor = torch.tensor(f_y_samples).float()
f_y_tensor.requires_grad = True
f_tensor = torch.tensor(f_samples).float()
f_tensor.requires_grad = True

In [66]:
class PINN(torch.nn.Module):
  def __init__(self):
    super(PINN,self).__init__()
    self.m1 = torch.nn.Linear(2,20)
    self.m2 = torch.nn.Linear(20,200)
    self.m3 = torch.nn.Linear(200,200)
    self.m4 = torch.nn.Linear(200,200)
    self.m5 = torch.nn.Linear(200,20)
    self.m6 = torch.nn.Linear(20,1)
    # change the number of layers or neurons if needed
  def forward(self,t,y):
    u = torch.cat((t.view(t.size(0), -1),y.view(y.size(0), -1)), dim=1)
    layer_1 = torch.tanh(self.m1(u))
    layer_2 = torch.tanh(self.m2(layer_1))
    layer_3 = torch.tanh(self.m3(layer_2))
    layer_4 = torch.tanh(self.m4(layer_3))
    layer_5 = torch.tanh(self.m5(layer_4))
    layer_6 = torch.tanh(self.m6(layer_5))
    # change the activation function if needed
    return layer_6

In [67]:
def f_map(u,t,y): # compute f from u
  u_t = nth_derivative(u.sum(), t, 1)
  u_yy = nth_derivative(u.sum(), y, 2)
  return u_t + dpdx/rho - nu*u_yy - g

In [68]:
def train(model,optimizer,criterion,epochs):
  for e in range(1,epochs+1):
    model.train()
    u_output = torch.reshape(model(u_t_tensor,u_y_tensor),(-1,))
    # MSE wrt u
    u_loss = criterion(u_output,u_tensor)
    u_temp = model(f_t_tensor,f_y_tensor)
    f_output = f_map(u_temp,f_t_tensor,f_y_tensor)
    # MSE wrt f
    f_loss = criterion(f_output,f_tensor)
    # change accorded weights
    loss = 0.67*u_loss + 0.33*f_loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f"| epoch {e:2d} | loss {loss:.6f}")

In [69]:
net = PINN()
optimizer = torch.optim.Adam(net.parameters(),lr=0.001)
criterion = torch.nn.MSELoss()
train(net,optimizer,criterion,10000)

Streaming output truncated to the last 5000 lines.
| epoch 5001 | loss 0.028158
| epoch 5002 | loss 0.025720
| epoch 5003 | loss 0.024237
| epoch 5004 | loss 0.022622
| epoch 5005 | loss 0.023495
| epoch 5006 | loss 0.025015
| epoch 5007 | loss 0.025273
| epoch 5008 | loss 0.025538
| epoch 5009 | loss 0.026694
| epoch 5010 | loss 0.026683
| epoch 5011 | loss 0.025047
| epoch 5012 | loss 0.024449
| epoch 5013 | loss 0.024316
| epoch 5014 | loss 0.024189
| epoch 5015 | loss 0.024456
| epoch 5016 | loss 0.026735
| epoch 5017 | loss 0.028446
| epoch 5018 | loss 0.030951
| epoch 5019 | loss 0.031407
| epoch 5020 | loss 0.032212
| epoch 5021 | loss 0.028412
| epoch 5022 | loss 0.025591
| epoch 5023 | loss 0.023935
| epoch 5024 | loss 0.024029
| epoch 5025 | loss 0.025102
| epoch 5026 | loss 0.026588
| epoch 5027 | loss 0.026591
| epoch 5028 | loss 0.024980
| epoch 5029 | loss 0.023169
| epoch 5030 | loss 0.021694
| epoch 5031 | loss 0.021005
| epoch 5032 | loss 0.021072
| epoch 5033 | loss 0

In [70]:
# animate the solution
M = 100
dy = h / M
T = 1.0
N = 100
dt = 1 / N
plt.ioff()
y = np.linspace(0, 1, num=M+1)
y_tensor = torch.from_numpy(y).float()
fig, ax = plt.subplots()
u_total = []
def animate(i):
  ax.cla()
  t = np.full_like(y_tensor,i*dt)
  t_tensor = torch.from_numpy(t).float()
  u_tensor = net(t_tensor,y_tensor)
  u_tensor = torch.reshape(u_tensor,(-1,))
  u = u_tensor.detach().numpy()
  u_total.append(u)
  ax.plot(u,y)
  ax.set_xlim(-4,4)
  ax.set_title("Time={}".format(round(i*dt,2)))
anime = animation.FuncAnimation(fig, animate, frames=N+1, interval=100)
anime.save('Numerical_solution.mp4', writer = "ffmpeg")
from google.colab import files
files.download('Numerical_solution.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [71]:
u_anal = []

In [72]:
if trivial == True:
  def u_analytic(t,y):
    C = -dpdx/rho + g
    r = 0
    for n in range(1,500):
      if n % 2 == 1:
        p1 = 4*C/(n**3*(np.pi)**3*nu) - 4*C/(n**3*(np.pi)**3*nu)*np.exp(-n**2*(np.pi**2)*nu*t)
        r += p1 * np.sin(n*np.pi*y)
    return r
  def animate_anal(j):
    ax.cla()
    t = np.full_like(y,j*dt)
    u = np.zeros(len(t))
    for k in range(len(t)):
      u[k] = u_analytic(t[k],y[k])
    u_anal.append(u)
    ax.plot(u,y)
    ax.set_xlim(-4,4)
    ax.set_title("Time={}".format(round(j*dt,2)))
  anime = animation.FuncAnimation(fig, animate_anal, frames=N+1, interval=100)
  anime.save('Analytical_solution.mp4', writer = "ffmpeg")
  files.download('Analytical_solution.mp4')

In [73]:
if constant == True:
  def u_analytic(t,y):
    C = -dpdx/rho + g
    r = 0
    gamma = 0.5 + 0.5*y
    for n in range(1,500):
      if n % 2 == 1:
        p = 4*C/(n**3*(np.pi)**3*nu) - (3/(n*np.pi)+4*C/(n**3*(np.pi)**3*nu))*np.exp(-n**2*(np.pi)**2*nu*t)
        r += p * np.sin(n*np.pi*y)
      else:
        p = np.exp(-n**2*(np.pi)**2*nu*t)/(n*np.pi)
        r += p * np.sin(n*np.pi*y)
    return r + gamma
  def animate_anal(j):
    ax.cla()
    t = np.full_like(y,j*dt)
    u = np.zeros(len(t))
    for k in range(len(t)):
      u[k] = u_analytic(t[k],y[k])
    u_anal.append(u)
    ax.plot(u,y)
    ax.set_xlim(-4,4)
    ax.set_title("Time={}".format(round(j*dt,2)))
  anime = animation.FuncAnimation(fig, animate_anal, frames=N+1, interval=100)
  anime.save('Analytical_solution.mp4', writer = "ffmpeg")
  files.download('Analytical_solution.mp4')

In [74]:
if sinusoidal == True:
  def u_analytic(t,y):
    C = -dpdx/rho + g
    k = 10
    gamma = (1-y) * np.sin(k*t) + y * np.cos(k*t)
    r = 0
    for n in range(1,500):
      den = n*np.pi*(n**4*(np.pi)**4*nu**2+k**2)
      if n % 2 == 1:
        s1 = 2*k*np.cos(k*t)*(n**2*(np.pi)**2*nu+k) / den
        s2 = 2*k*np.sin(k*t)*((n**2)*(np.pi)**2*nu-k) / den
        s3 = 4*C/(n**3*(np.pi)**3*nu)
        p1 = -s1 + s2 + s3
        p2 = -2/(n*np.pi) + 2*k*(n**2*(np.pi)**2*nu+k) / den - s3
        p3 = p2 * np.exp(-n**2*(np.pi)**2*nu*t)
        u = (p1 + p3) * np.sin(n*np.pi*y)
        r += u
      else:
        s1 = 2*k*np.cos(k*t)*(n**2*(np.pi)**2*nu-k) / den
        s2 = 2*k*np.sin(k*t)*(n**2*(np.pi)**2*nu+k) / den
        p1 = -s1 - s2
        p2 = 2/(n*np.pi) + 2*k*(n**2*(np.pi)**2*nu-k) / den
        p3 = p2 * np.exp(-n**2*(np.pi)**2*nu*t)
        u = (p1 + p3) * np.sin(n*np.pi*y)
        r += u
    result = gamma + r
    return result
  def animate_anal(j):
    ax.cla()
    t = np.full_like(y,j*dt)
    u = np.zeros(len(t))
    for k in range(len(t)):
      u[k] = u_analytic(t[k],y[k])
    u_anal.append(u)
    ax.plot(u,y)
    ax.set_xlim(-4,4)
    ax.set_title("Time={}".format(round(j*dt,2)))
  anime = animation.FuncAnimation(fig, animate_anal, frames=N+1, interval=100)
  anime.save('Analytical_solution.mp4', writer = "ffmpeg")
  files.download('Analytical_solution.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [75]:
err = []
for i in range(len(u_total)):
  diff = u_total[i] - u_anal[i]
  d = np.sqrt(dy * np.dot(diff, diff))
  err.append(d)
err = np.array(err)
print("Average error:", np.mean(err))

Average error: 0.08674401995775581
